In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
import datetime

In [5]:
road = 7
direction = "in"
month = "03"
df = pd.read_csv(
    f"../dataset/2020-{month}-road-{road}-{direction}.csv", parse_dates=["datetime"])
df


,datetime,road_number,km,direction,all_units,inflow_units,outflow_unit,samecell_units,avg_speed,max_speed,avg_traveltime,max_traveltime
0,2020-03-01 00:00:21,7,2,in,1,1,0,0,76,76,45,45
1,2020-03-01 00:00:21,7,3,in,1,1,0,0,105,105,30,30
2,2020-03-01 00:00:21,7,4,in,0,0,1,0,0,0,0,0
3,2020-03-01 00:00:21,7,5,in,2,2,1,0,106,112,59,60
4,2020-03-01 00:00:21,7,6,in,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5066350,2020-03-31 23:59:46,7,113,in,1,1,0,0,69,69,60,60
5066351,2020-03-31 23:59:46,7,114,in,1,0,0,1,0,0,0,0
5066352,2020-03-31 23:59:46,7,115,in,1,0,1,1,61,61,0,0
5066353,2020-03-31 23:59:46,7,116,in,2,2,0,0,55,58,59,59


In [ ]:
df_km127 = pd.read_csv("../dataset/latlon_km127.csv")

In [ ]:
import joblib
dtree_model = joblib.load('../Models/DecisionTreeModelRoad7in.joblib')

In [ ]:
dtree_model

In [ ]:
def map_traffic_with_latlon(df):
    df['lat'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values) > 0 else 0, axis=1)
    df['lon'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values) > 0 else 0, axis=1)
    return df


In [ ]:
header_list = ['datetime', 'road_number', 'km', 'direction', 'all_units', 'inflow_units',
               'outflow_unit', 'samecell_units', 'avg_speed', 'max_speed', 'avg_traveltime', 'max_traveltime']
df_test = pd.read_csv("../dataset/current_celldata.csv",
                         names=header_list, parse_dates=["datetime"])
df_test = df_test[(df_test['road_number'] == 1) | (
    df_test['road_number'] == 2) | (df_test['road_number'] == 7)]
df_test = map_traffic_with_latlon(df_test)
df_test


In [ ]:
num_cols = ['all_units', 'inflow_units',
            'avg_speed', 'max_speed',
            'max_traveltime']
num_cols

In [ ]:
x_predict = df_1_out[num_cols]
x_predict

In [ ]:
y_predict = dtree_model.predict(x_predict)
y_predict

In [ ]:
df["cluster"] = y_predict
df

In [ ]:
df_acc = df[((df["cluster"] == 1) | (df["cluster"] == 4)) & (df["datetime"] > pd.Timestamp(2020, 2, 28, 21, 00))
                  & (df["datetime"] < pd.Timestamp(2020, 2, 28, 21, 10))]
df_acc


In [ ]:
df_acc[(df_acc["km"] == 47)]
